In [6]:
!pip install transformers torch datasets
from transformers import BertTokenizer, BertModel, pipeline, AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments, AutoModelForQuestionAnswering
from datasets import load_dataset, Dataset
import torch
import torch.nn.functional as F
import re
from nltk.tokenize import sent_tokenize
import nltk
import json
import os
import numpy as np
os.environ["WANDB_DISABLED"] = "true"
nltk.download('punkt')
nltk.download('punkt_tab')
from google.colab import drive
drive.mount('/content/drive')



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Mounted at /content/drive


In [22]:
def load_text(file_path):
    """
    Load text from a file.
    """
    print("Loading the text from the file...")
    with open(book_path, 'r', encoding='utf-8') as file:
        return file.read()

def remove_gutenberg_header_footer(text, start_marker, end_marker):
    """
    Remove the header and footer from Project Gutenberg text.

    Finds specified start and end markers and returns the content in between.

    Parameters:
        text (str): The text to clean.
        start_marker (str): Marker indicating the start of main content.
        end_marker (str): Marker indicating the end of main content.

    Returns:
        str: Text with header and footer removed, or original text if markers are not found.
    """
    start_index = text.find(start_marker)
    if start_index == -1:
        return text

    end_index = text.find(end_marker)
    if end_index == -1:
        return text

    cleaned_text = text[start_index + len(start_marker):end_index].strip()
    return cleaned_text

book_path = "/content/drive/My Drive/Cosc524 - Collaboration/a study in scarlet.txt"

# here we are cleaning the text and removing the headers and footers
raw_text = load_text(book_path)

start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK A STUDY IN SCARLET ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK A STUDY IN SCARLET ***"

cleaned_text = remove_gutenberg_header_footer(raw_text, start_marker, end_marker)

print("Cleaned text sample:\n")
print(cleaned_text[:500])

def tokenize_sentences(text):
    """
    Tokenize the input text into sentences using NLTK.

    Parameters:
        text (str): The text to tokenize.

    Returns:
        list: A list of tokenized sentences.
    """
    print("Tokenizing the text into sentences...")
    sentences = sent_tokenize(text)
    print(f"Total sentences: {len(sentences)}")
    return sentences

my_sentences = tokenize_sentences(cleaned_text)

Loading the text from the file...
Cleaned text sample:

A STUDY IN SCARLET

By A. Conan Doyle




CONTENTS

 A STUDY IN SCARLET.

 PART I.
 CHAPTER I. MR. SHERLOCK HOLMES.
 CHAPTER II. THE SCIENCE OF DEDUCTION.
 CHAPTER III. THE LAURISTON GARDENS MYSTERY
 CHAPTER IV. WHAT JOHN RANCE HAD TO TELL.
 CHAPTER V. OUR ADVERTISEMENT BRINGS A VISITOR.
 CHAPTER VI. TOBIAS GREGSON SHOWS WHAT HE CAN DO.
 CHAPTER VII. LIGHT IN THE DARKNESS.

 PART II. THE COUNTRY OF THE SAINTS
 CHAPTER I. ON THE GREAT ALKALI PLAIN.
 CHAPTER II. THE FLOWER OF UTAH.
 CHAPTER III. J
Tokenizing the text into sentences...
Total sentences: 2208
Number of tokenized sentences: 2208

Sample tokenized sentences:
1: A STUDY IN SCARLET

By A. Conan Doyle




CONTENTS

 A STUDY IN SCARLET.
2: PART I.
3: CHAPTER I. MR. SHERLOCK HOLMES.
4: CHAPTER II.
5: THE SCIENCE OF DEDUCTION.


In [11]:
# For text generation
generator = pipeline('fill-mask', model='bert-base-uncased')

# For text retrieval
retriever_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
retriever_model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
def retrieve_relevant_info(query, top_k=5, embeddings=None, sentences=None):

  """Retrieves the most relevant sentences based on cosine similarity.

  Args:
    query: The input query string.
    top_k: The number of top sentences to retrieve.

  Returns:
    A list of the top_k most relevant sentences.
  """
  query_embedding = retriever_model(**retriever_tokenizer(query, return_tensors="pt")).last_hidden_state[:, 0, :]
  similarities = F.cosine_similarity(embeddings, query_embedding, dim=1) # Assume 'embeddings' is pre-calculated
  top_indices = torch.topk(similarities, top_k).indices
  relevant_sentences = [sentences[idx] for idx in top_indices]  # Assume 'sentences' is pre-calculated
  return relevant_sentences

In [26]:
def generate_text_with_rag(prompt, top_k=5, embeddings=None, sentences=None):
  """Generates text using RAG with BERT.

  Args:
    prompt: The input prompt for text generation.
    top_k: The number of retrieved sentences to use for augmentation.

  Returns:
    The generated text.
  """
  relevant_info = retrieve_relevant_info(prompt, top_k=top_k, embeddings=embeddings, sentences=sentences)
  augmented_prompt = f"{prompt} [SEP] {' '.join(relevant_info)} [MASK]."  # Add [MASK] token
  generated_text = generator(augmented_prompt)[0]['sequence']
  return generated_text

In [29]:
my_embeddings = torch.load('/content/drive/My Drive/Cosc524 - Collaboration/embeddings/bert_embeddings.pt')
prompt = "Who was the murderer?"
generated_text = generate_text_with_rag(prompt=prompt, top_k=5, embeddings=my_embeddings, sentences=my_sentences)
# print(generated_text)
for sentence in sent_tokenize(generated_text):
  print(sentence)

<ipython-input-29-90129c60a95d>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  my_embeddings = torch.load('/content/drive/My Drive/Cosc524 - Collaboration/embeddings/bert_

who was the murderer?
where did the blood come from?
why should he fear a trap?
but why should he come back to the house after leaving it?
what was that?
how came the woman ’ s ring there?
no.
